<a href="https://colab.research.google.com/github/rhiats/Heart-Failure-Clinical-Records/blob/main/Heart_Failure_Clinical_Records.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Heart Failure Clinical Records**

Replicate paper:

Chicco, D., & Jurman, G. (2020). Machine learning can predict survival of patients with heart failure from serum creatinine and ejection fraction alone. BMC Medical Informatics and Decision Making, 20, 16. https://doi.org/10.1186/s12911-020-1023-5

Dataset:
Heart Failure Clinical Records [Dataset]. (2020). UCI Machine Learning Repository. https://doi.org/10.24432/C5Z89R.

In [1]:
!pip install ucimlrepo

**Load Data from UCI ML Repo**

In [2]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
heart_failure_clinical_records = fetch_ucirepo(id=519)

# data (as pandas dataframes)
X = heart_failure_clinical_records.data.features
y = heart_failure_clinical_records.data.targets

# metadata
print(heart_failure_clinical_records.metadata)

# variable information
print(heart_failure_clinical_records.variables)

{'uci_id': 519, 'name': 'Heart Failure Clinical Records', 'repository_url': 'https://archive.ics.uci.edu/dataset/519/heart+failure+clinical+records', 'data_url': 'https://archive.ics.uci.edu/static/public/519/data.csv', 'abstract': 'This dataset contains the medical records of 299 patients who had heart failure, collected during their follow-up period, where each patient profile has 13 clinical features.', 'area': 'Health and Medicine', 'tasks': ['Classification', 'Regression', 'Clustering'], 'characteristics': ['Multivariate'], 'num_instances': 299, 'num_features': 12, 'feature_types': ['Integer', 'Real'], 'demographics': ['Age', 'Sex'], 'target_col': ['death_event'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2020, 'last_updated': 'Mon Feb 26 2024', 'dataset_doi': '10.24432/C5Z89R', 'creators': [], 'intro_paper': {'ID': 286, 'type': 'NATIVE', 'title': 'Machine learning can predict survival of patients with heart failure f

**Continuous Features EDA**

**Categorical Features EDA**

**Mann–Whitney U test**

**Pearson correlation coefficient**

**Scatterplot comparing Serum Creatine v. Ejection Fraction**

**Survival Month**

**Survival Month v. Survival**

**Survival Prediction on all Clinical Features**



1. Random Forests
2. Decision Tree
3. Gradient Boosting
4. One Rule
5. Artificial Neural Network
6. Naive Bayes
7. SVM Radial
8. SVM Linear
9. K-nearest Neighbor

Found Random Forest performed the best.







**Feature Ranking**


1. RReliefF
2. Max-Min Parents and Children
3. Random Forest
4. One Rule
5. Recursive Partitioning and Regression Trees
6. Support Vector Machines with linear kernel
7. eXtreme Gradient Boosting

11 features (everything but Time)

Borda list: lower the score,the more important the feature

The authors found the top–two features to be serum
creatinine and ejection fraction. They used these 2 features in 3 classifiers:
Random Forests (RF),
Support Vector Machine with Gaussian Kernel (GSVM)
and eXtreme Gradient Boosting (XGB).


**Random Forests (RF)**

**Support Vector Machine with Gaussian Kernel (GSVM)**

**eXtreme Gradient Boosting (XGB)**

**Stratified Logistic Regression All Features**

**Stratified Logistic Regression Top 3 Features**